# CIE 5104 Traveler Behavior Analysis: Mid-Term Examination (Take-Home)
## Q1 Box office prediction
### Data collection - TMDB

In [1]:
import pandas as pd
import requests
from bs4 import BeautifulSoup
import numpy as np
from sklearn.model_selection import train_test_split
import math
from collections import Counter
from datetime import datetime
from xgboost import XGBRegressor
from sklearn.multioutput import MultiOutputRegressor
import re
from sklearn.model_selection import GridSearchCV, train_test_split

In [2]:
from google.colab import drive
drive.mount('/content/drive')

Mounted at /content/drive


In [17]:
df = pd.read_csv('/content/drive/MyDrive/movies_views.csv').dropna(subset=['opening_box_office'])
df['keywords'].fillna('[]', inplace=True)
df.head()

,adult,belongs_to_collection,budget,genres,id,imdb_id,original_language,original_title,popularity,production_companies,production_countries,release_date,runtime,spoken_languages,title,opening_box_office,casts,keywords,trailer_views
0,False,NaN,4000000.0,"[{'id': 80, 'name': 'Crime'}, {'id': 35, 'name...",5.0,tt0113101,en,Four Rooms,18.437,"[{'id': 14, 'logo_path': '/m6AHu84oZQxvq7n1rsv...","[{'iso_3166_1': 'US', 'name': 'United States o...",1995-12-09,98.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Four Rooms,427733.0,"[{'adult': False, 'gender': 2, 'id': 3129, 'kn...","[{'id': 612, 'name': 'hotel'}, {'id': 613, 'na...",NaN
1,False,NaN,21000000.0,"[{'id': 28, 'name': 'Action'}, {'id': 53, 'nam...",6.0,tt0107286,en,Judgment Night,10.566,"[{'id': 33, 'logo_path': '/8lvHyhjr8oUKOOy2dKX...","[{'iso_3166_1': 'JP', 'name': 'Japan'}, {'iso_...",1993-10-15,110.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Judgment Night,4088955.0,"[{'adult': False, 'gender': 2, 'id': 2880, 'kn...","[{'id': 520, 'name': 'chicago, illinois'}, {'i...",NaN
2,False,"{'id': 10, 'name': 'Star Wars Collection', 'po...",11000000.0,"[{'id': 12, 'name': 'Adventure'}, {'id': 28, '...",11.0,tt0076759,en,Star Wars,89.361,"[{'id': 1, 'logo_path': '/o86DbpburjxrqAzEDhXZ...","[{'iso_3166_1': 'US', 'name': 'United States o...",1977-05-25,121.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Star Wars,1554475.0,"[{'adult': False, 'gender': 2, 'id': 2, 'known...","[{'id': 803, 'name': 'android'}, {'id': 4270, ...",NaN
3,False,"{'id': 137697, 'name': 'Finding Nemo Collectio...",94000000.0,"[{'id': 16, 'name': 'Animation'}, {'id': 10751...",12.0,tt0266543,en,Finding Nemo,132.131,"[{'id': 3, 'logo_path': '/1TjvGVDMYsj6JBxOAkUH...","[{'iso_3166_1': 'US', 'name': 'United States o...",2003-05-30,100.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Finding Nemo,70251710.0,"[{'adult': False, 'gender': 2, 'id': 13, 'know...","[{'id': 970, 'name': 'parent child relationshi...",NaN
4,False,NaN,55000000.0,"[{'id': 35, 'name': 'Comedy'}, {'id': 18, 'nam...",13.0,tt0109830,en,Forrest Gump,67.124,"[{'id': 4, 'logo_path': '/fycMZt242LVjagMByZOL...","[{'iso_3166_1': 'US', 'name': 'United States o...",1994-07-06,142.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Forrest Gump,24450602.0,"[{'adult': False, 'gender': 2, 'id': 31, 'know...","[{'id': 422, 'name': 'vietnam veteran'}, {'id'...",NaN


In [19]:
pred = pd.read_csv('/content/drive/MyDrive/pred.csv')
pred['trailer_views'] = 88330000.0
pred['no_trailer'] = 0
pred

,adult,belongs_to_collection,budget,genres,id,imdb_id,original_language,original_title,popularity,production_companies,production_countries,release_date,runtime,spoken_languages,title,casts,keywords,trailer_views,no_trailer
0,False,"{'id': 618529, 'name': 'Doctor Strange Collect...",170000000,"[{'id': 14, 'name': 'Fantasy'}, {'id': 28, 'na...",453395,tt9419884,en,Doctor Strange in the Multiverse of Madness,519.885,"[{'id': 420, 'logo_path': '/hUzeosd33nzE5MCNsZ...","[{'iso_3166_1': 'US', 'name': 'United States o...",2022-05-04,126.0,"[{'english_name': 'English', 'iso_639_1': 'en'...",Doctor Strange in the Multiverse of Madness,"[{'adult': False, 'gender': 2, 'id': 71580, 'k...","[{'id': 2343, 'name': 'magic'}, {'id': 9717, '...",88330000.0,0


In [5]:
list_of_genres = sum(list(df['genres'].apply(lambda x: [i['name'] for i in eval(x)])), [])
top_genres = [m[0] for m in Counter(list_of_genres).most_common(15)]

list_of_production_companies = sum(list(df['production_companies'].apply(lambda x: [i['name'] for i in eval(x)])), [])
top_production_companies = [m[0] for m in Counter(list_of_production_companies).most_common(30)]

list_of_production_countries = sum(list(df['production_countries'].apply(lambda x: [i['name'] for i in eval(x)])), [])
top_production_countries = [m[0] for m in Counter(list_of_production_countries).most_common(25)]

list_of_spoken_languages = sum(list(df['spoken_languages'].apply(lambda x: [i['english_name'] for i in eval(x)])), [])
top_spoken_languages = [m[0] for m in Counter(list_of_spoken_languages).most_common(15)]

list_of_keywords = sum(list(df['keywords'].apply(lambda x: [i['name'] for i in eval(x)])), [])
top_keywords = [m[0] for m in Counter(list_of_keywords).most_common(30)]

list_of_casts_names = sum(list(df['casts'].apply(lambda x: [i['name'] for i in eval(x)])), [])
top_casts_names = [m[0] for m in Counter(list_of_casts_names).most_common(15)]

list_of_casts_characters = sum(list(df['casts'].apply(lambda x: [i['character'] for i in eval(x)])), [])
top_casts_characters = [m[0] for m in Counter(list_of_casts_characters).most_common(15)]

In [20]:
def collection_preprocessing(x):
    x = str(x)
    if x == 'nan':
        return -1
    else:
        return int(eval(x)['id'])
        
def get_days_diff(x):
    date_format = "%Y-%m-%d"
    date_base = datetime.strptime('2022-05-04', date_format)
    date_target = datetime.strptime(x, date_format)
    return (date_target - date_base).days

def get_month(x):
    date_format = "%Y-%m-%d"
    date_target = datetime.strptime(x, date_format)
    return date_target.month
    

def pre_processing(X):
    X.drop(['id', 'imdb_id'], axis=1, inplace=True)
    X['genres'] = X['genres'].apply(lambda x: eval(x))
    X['production_companies'] = X['production_companies'].apply(lambda x: eval(x))
    X['production_countries'] = X['production_countries'].apply(lambda x: eval(x))
    X['spoken_languages'] = X['spoken_languages'].apply(lambda x: eval(x))
    X['casts'] = X['casts'].apply(lambda x: eval(x))
    X['keywords'] = X['keywords'].apply(lambda x: eval(x))

    X['collection_id'] = X['belongs_to_collection'].apply(collection_preprocessing)
    X['has_collection'] = X['collection_id'].apply(lambda x: 1 if x != '' else 0)
    X = X.drop(['belongs_to_collection'], axis=1)

    X['genres_list'] = X['genres'].apply(lambda x: [i['name'] for i in x])
    X['num_genres'] = X['genres_list'].apply(len)
    for g in top_genres:
        X['genre_' + g] = X['genres_list'].apply(lambda x: 1 if g in x else 0)
    X = X.drop(['genres', 'genres_list'], axis=1)
    
    X['production_companies_list'] = X['production_companies'].apply(lambda x: [i['name'] for i in x])
    X['num_production_companies'] = X['production_companies_list'].apply(len)
    for c in top_production_companies:
        X['production_company_' + c] = X['production_companies_list'].apply(lambda x: 1 if c in x else 0)
    X = X.drop(['production_companies', 'production_companies_list'], axis=1)

    X['production_countries_list'] = X['production_countries'].apply(lambda x: [i['name'] for i in x])
    X['num_production_countries'] = X['production_countries_list'].apply(len)
    for c in top_production_countries:
        X['production_country_' + c] = X['production_countries_list'].apply(lambda x: 1 if c in x else 0)
    X = X.drop(['production_countries', 'production_countries_list'], axis=1)

    X['spoken_languages_list'] = X['spoken_languages'].apply(lambda x: [i['english_name'] for i in x])
    X['num_spoken_languages'] = X['spoken_languages_list'].apply(len)
    for l in top_spoken_languages:
        X['spoken_language_' + l] = X['spoken_languages_list'].apply(lambda x: 1 if l in x else 0)
    X = X.drop(['spoken_languages', 'spoken_languages_list'], axis=1)

    X['keywords_list'] = X['keywords'].apply(lambda x: [i['name'] for i in x])
    X['num_keywords'] = X['keywords_list'].apply(len)
    for k in top_keywords:
        X['keyword_' + k] = X['keywords_list'].apply(lambda x: 1 if k in x else 0)
    X = X.drop(['keywords', 'keywords_list'], axis=1)
    
    X['casts_list'] = X['casts'].apply(lambda x: [i['name'] for i in x])
    X['num_casts'] = X['casts_list'].apply(len)
    for c in top_casts_names:
        X['cast_' + c] = X['casts_list'].apply(lambda x: 1 if c in x else 0)
    for c in top_casts_characters:
        X['cast_character_' + c] = X['casts_list'].apply(lambda x: 1 if c in x else 0)
    X['casts_gender_0'] = X['casts'].apply(lambda x: sum([1 for i in x if ['gender'] == 0]))
    X['casts_gender_1'] = X['casts'].apply(lambda x: sum([1 for i in x if ['gender'] == 1]))
    X['casts_gender_2'] = X['casts'].apply(lambda x: sum([1 for i in x if ['gender'] == 2]))    
    X = X.drop(['casts', 'casts_list'], axis=1)

    X['log_budget'] = np.log1p(X['budget'])

    X['days_before'] = X['release_date'].apply(get_days_diff)
    X['month'] = X['release_date'].apply(get_month)
    for m in range(1,13):
      X[f'month_{m}'] = X['month'].apply(lambda x: 1 if m == x else 0)
    X = X.drop(['release_date', 'month'], axis=1)

    X['adult'] = X['adult'].apply(lambda x: 1 if x == 'True' else 0)
    X['adult'] = X['adult'].astype(np.int64)

    X['no_trailer'] = X['trailer_views'].apply(lambda x: 1 if np.isnan(x) else 0)
    X['trailer_views'] = X['trailer_views'].fillna(0)

    X = X.drop(['original_language', 'original_title', 'title'], axis=1)
    X = X.rename(columns = lambda x:re.sub('[^A-Za-z0-9_]+', '', x))
    X.fillna(0)
    return X

In [93]:
X = df.drop(['opening_box_office'], axis=1)
y = df['opening_box_office']
X = pre_processing(X)
gsc = GridSearchCV(
        estimator=XGBRegressor(),
        param_grid={"learning_rate": (0.05, 0.10, 0.15),
                    "max_depth": [ 3, 4, 5, 6, 8],
                    "min_child_weight": [ 1, 3, 5, 7],
                    "gamma":[ 0.0, 0.1, 0.2],
                    "colsample_bytree":[ 0.3, 0.4],},
        cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
grid_result = gsc.fit(X, y)
print(f'best score: {grid_result.best_score_}')
print(f'prediction: {grid_result.best_estimator_.predict(pred)[0]}')

[09:01:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best score: -132140320428685.75
prediction: 51286580.0


In [94]:
df_1k = df.nlargest(1000, 'opening_box_office')
X = df_1k.drop(['opening_box_office'], axis=1)
y = df_1k['opening_box_office']
X = pre_processing(X)
gsc = GridSearchCV(
        estimator=XGBRegressor(),
        param_grid={"learning_rate": (0.05, 0.10, 0.15),
                    "max_depth": [ 3, 4, 5, 6, 8],
                    "min_child_weight": [ 1, 3, 5, 7],
                    "gamma":[ 0.0, 0.1, 0.2],
                    "colsample_bytree":[ 0.3, 0.4],},
        cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
grid_result = gsc.fit(X, y)
print(f'best score: {grid_result.best_score_}')
print(f'prediction: {grid_result.best_estimator_.predict(pred)[0]}')

[09:04:31] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best score: -1486398820843763.2
prediction: 55637892.0


In [95]:
df_2k = df.nlargest(2000, 'opening_box_office')
X = df_2k.drop(['opening_box_office'], axis=1)
y = df_2k['opening_box_office']
X = pre_processing(X)
gsc = GridSearchCV(
        estimator=XGBRegressor(),
        param_grid={"learning_rate": (0.05, 0.10, 0.15),
                    "max_depth": [ 3, 4, 5, 6, 8],
                    "min_child_weight": [ 1, 3, 5, 7],
                    "gamma":[ 0.0, 0.1, 0.2],
                    "colsample_bytree":[ 0.3, 0.4],},
        cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
grid_result = gsc.fit(X, y)
print(f'best score: {grid_result.best_score_}')
print(f'prediction: {grid_result.best_estimator_.predict(pred)[0]}')

[09:09:58] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best score: -966778448299241.4
prediction: 61067728.0


In [96]:
df_5h = df.nlargest(500, 'opening_box_office')
X = df_5h.drop(['opening_box_office'], axis=1)
y = df_5h['opening_box_office']
X = pre_processing(X)
gsc = GridSearchCV(
        estimator=XGBRegressor(),
        param_grid={"learning_rate": (0.05, 0.10, 0.15),
                    "max_depth": [ 3, 4, 5, 6, 8],
                    "min_child_weight": [ 1, 3, 5, 7],
                    "gamma":[ 0.0, 0.1, 0.2],
                    "colsample_bytree":[ 0.3, 0.4],},
        cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
grid_result = gsc.fit(X, y)
print(f'best score: {grid_result.best_score_}')
print(f'prediction: {grid_result.best_estimator_.predict(pred)[0]}')

[09:11:41] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best score: -2048943050524630.2
prediction: 68926976.0


In [32]:
df_3h = df.nlargest(300, 'opening_box_office')
X = df_3h.drop(['opening_box_office'], axis=1)
y = df_3h['opening_box_office'].apply(np.log1p)
X = pre_processing(X)
# pred_ = pre_processing(pred)
gsc = GridSearchCV(
        estimator=XGBRegressor(),
        param_grid={"learning_rate": (0.05, 0.10, 0.15),
                    "max_depth": [ 3, 4, 5, 6, 8],
                    "min_child_weight": [ 1, 3, 5, 7],
                    "gamma":[ 0.0, 0.1, 0.2],
                    "colsample_bytree":[ 0.3, 0.4],},
        cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
grid_result = gsc.fit(X, y)
print(f'best score: {grid_result.best_score_}')
print(f'prediction: {np.expm1(grid_result.best_estimator_.predict(pred_)[0])}')

prediction: 79310488.0


In [29]:
df_1h = df.nlargest(100, 'opening_box_office')
X = df_1h.drop(['opening_box_office'], axis=1)
y = df_1h['opening_box_office']
X = pre_processing(X)
# pred_ = pre_processing(pred)
# pred_ = pred_[X.columns]
gsc = GridSearchCV(
        estimator=XGBRegressor(),
        param_grid={"learning_rate": (0.05, 0.10, 0.15),
                    "max_depth": [ 3, 4, 5, 6, 8],
                    "min_child_weight": [ 1, 3, 5, 7],
                    "gamma":[ 0.0, 0.1, 0.2],
                    "colsample_bytree":[ 0.3, 0.4],},
        cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
grid_result = gsc.fit(X, y)
print(f'best score: {grid_result.best_score_}')
print(f'prediction: {grid_result.best_estimator_.predict(pred_)[0]}')

[12:59:18] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best score: -3699719964045074.0
prediction: 110521280.0


In [43]:
fi = grid_result.best_estimator_.feature_importances_
df_fi = pd.DataFrame(columns = ['feature', 'value'])
df_fi['feature'] = X.columns
df_fi['value'] = fi
df_fi.sort_values(by=['value'], ascending=False)

,feature,value
121,keyword_basedoncomic,0.079653
1,budget,0.071764
129,cast_SamuelLJackson,0.055552
166,month_4,0.048387
161,log_budget,0.039923
...,...,...
64,production_country_China,0.000000
63,production_country_Spain,0.000000
62,production_country_Ireland,0.000000
61,production_country_Japan,0.000000


In [33]:
grid_result.best_params_

{'colsample_bytree': 0.4,
 'gamma': 0.0,
 'learning_rate': 0.15,
 'max_depth': 3,
 'min_child_weight': 7}

In [98]:
df_3k = df.nlargest(3000, 'opening_box_office')
X = df_3k.drop(['opening_box_office'], axis=1)
y = df_3k['opening_box_office']
X = pre_processing(X)
gsc = GridSearchCV(
        estimator=XGBRegressor(),
        param_grid={"learning_rate": (0.05, 0.10, 0.15),
                    "max_depth": [ 3, 4, 5, 6, 8],
                    "min_child_weight": [ 1, 3, 5, 7],
                    "gamma":[ 0.0, 0.1, 0.2],
                    "colsample_bytree":[ 0.3, 0.4],},
        cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)
grid_result = gsc.fit(X, y)
print(f'best score: {grid_result.best_score_}')
print(f'prediction: {grid_result.best_estimator_.predict(pred)[0]}')

[09:20:48] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.
best score: -751308815544494.9
prediction: 45948292.0


In [90]:
gsc = GridSearchCV(
            estimator=XGBRegressor(),
            param_grid={"learning_rate": (0.05, 0.10, 0.15),
                        "max_depth": [ 3, 4, 5, 6, 8],
                        "min_child_weight": [ 1, 3, 5, 7],
                        "gamma":[ 0.0, 0.1, 0.2],
                        "colsample_bytree":[ 0.3, 0.4],},
            cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

grid_result = gsc.fit(train_X, train_y)

[08:39:51] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [91]:
grid_result.best_score_

-105884401528764.12

In [92]:
grid_result.best_estimator_.predict(pred)

array([54806516.], dtype=float32)

In [83]:
df_top1k = df.nlargest(1000, 'opening_box_office')

In [84]:
train_X_top1k, test_X_top1k, train_y_top1k, test_y_top1k = train_test_split(
    df_top1k.drop(['opening_box_office'], axis=1), df_top1k['opening_box_office'],
    test_size=0.2, random_state=7414)

train_X_top1k = pre_processing(train_X_top1k)
test_X_top1k = pre_processing(test_X_top1k)

In [87]:
gsc = GridSearchCV(
            estimator=XGBRegressor(),
            param_grid={"learning_rate": (0.05, 0.10, 0.15),
                        "max_depth": [ 3, 4, 5, 6, 8],
                        "min_child_weight": [ 1, 3, 5, 7],
                        "gamma":[ 0.0, 0.1, 0.2],
                        "colsample_bytree":[ 0.3, 0.4],},
            cv=3, scoring='neg_mean_squared_error', verbose=0, n_jobs=-1)

grid_result_top1k = gsc.fit(train_X, train_y)

[08:22:33] WARNING: /workspace/src/objective/regression_obj.cu:152: reg:linear is now deprecated in favor of reg:squarederror.


In [88]:
grid_result_top1k.best_score_

-105884401528764.12

In [89]:
grid_result_top1k.best_estimator_.predict(pred)

array([54806516.], dtype=float32)